In [1]:
import calendar
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine
from math import ceil, floor, trunc

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
con_lite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.float_format','{:,.2f}'.format)
pd.set_option('max_rows',None)

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

today = date.today()
today

datetime.date(2021, 12, 29)

### Today = Last Closed Work Day

In [2]:
#today = today - timedelta(days=3)
today

datetime.date(2021, 12, 29)

### Restart and run all cells

In [3]:
def number_of_spread(x):
    maxp = x[1]
    minp = x[5]
    div = 0.0
    gap = 0.0
    spd = 0
    spd1 = 0
    spd2 = 0
    if (maxp > minp): 
        mult = 1
    else:
    	tmpp = minp
    	minp = maxp
    	maxp = tmpp
    	mult = -1
        
    gap = round(maxp - minp,2)   
    
    if (maxp > 400.00):
        div = 2.0
        boundary = div * 200
        if ((minp < boundary)):
            spd1 = trunc((boundary - minp) / 1.00)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div) 
    elif (maxp > 200.00):
        div = 1.0
        boundary = div * 200
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.50)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div) 
    elif (maxp > 100.00):
        div = 0.50
        boundary = div * 200
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.25)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div) 
    elif (maxp > 25.00):
        div = 0.25
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.10)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div)
    elif (maxp > 10.00):
        div = 0.10
        boundary = div * 100
        if (minp < boundary):
            spd1 = ceil((boundary - minp) / 0.05)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            div = 0.10            
            spd = ceil(gap / div)
    elif (maxp > 5.00):
        div = 0.05
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.02)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2           
        else:
            div = 0.05            
            spd = ceil(gap / div)
    elif (maxp > 2.00):
        div = 0.02
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.01)
            spd2 = trunc((maxp - boundary) / div)
            spd = spd1 + spd2
        else:          
            spd = (gap / div)
    else:
        div = 0.01
        boundary = div * 100        
        spd = trunc(gap / div) 
        
    return spd * mult

In [4]:
format_dict = {
               'fm_price':'{:.2f}','to_price':'{:.2f}','price':'{:.2f}',    
               'max_price':'{:.2f}','min_price':'{:.2f}',
               'target':'{:.2f}','max':'{:.2f}','max52':'{:.2f}',
                'min':'{:.2f}','min52':'{:.2f}',
    'css_price':'{:.2f}','amount':'{:,.2f}',  
               'unit_cost':'{:.2f}','cost_amt':'{:,.2f}',
               'dividend':'{:.2f}','dividend_amount':'{:,.2f}',    
               'cost_amt':'{:,.2f}','div_amt':'{:,.2f}','cost_pct':'{:.2f}', 
               'buy_target':'{:.2f}','sell_target':'{:.2f}',
               'volume':'{:,.2f}','beta':'{:.2f}','cost':'{:,.2f}',
               'qty':'{:,}','available_qty':'{:,}','shares':'{:,}'}
pd.set_option("display.max_rows", None)

In [5]:
sql = '''
SELECT T.name,buy_target AS target, bl, T.qty,
fm_price,to_price,  ttl_spread AS ttl, A.min_price AS min, 
T.min_price AS min52, consensus_price AS css_price, buy_target * T.qty AS amount, T.status,
FM_date, to_date, days
FROM stocks T
JOIN sales A
ON T.name = A.name
LEFT JOIN consensus C
ON T.name = C.name
WHERE T.status IN ('B','O')
AND buy_target > 0
AND to_date = '%s'
'''
sql = sql % today
print(sql)


SELECT T.name,buy_target AS target, bl, T.qty,
fm_price,to_price,  ttl_spread AS ttl, A.min_price AS min, 
T.min_price AS min52, consensus_price AS css_price, buy_target * T.qty AS amount, T.status,
FM_date, to_date, days
FROM stocks T
JOIN sales A
ON T.name = A.name
LEFT JOIN consensus C
ON T.name = C.name
WHERE T.status IN ('B','O')
AND buy_target > 0
AND to_date = '2021-12-29'



In [6]:
df_stock = pd.read_sql(sql, con_lite)
#df_stock.sort_values(['name'],ascending=[True])
df_stock.sort_values(['name'],ascending=[True]).style.format(format_dict)

,name,target,bl,qty,fm_price,to_price,ttl,min,min52,css_price,amount,status,fm_date,to_date,days
0,DOHOME,22.90,1,0,23.90,23.20,-7,23.10,12.37,30.91,0.00,B,2021-12-22,2021-12-29,5
1,GLOBAL,19.00,0,0,20.00,20.20,2,20.10,15.69,26.25,0.00,B,2021-12-23,2021-12-29,4
7,NOBLE,5.85,0,0,6.10,6.15,1,6.10,5.90,6.80,0.00,B,2021-12-28,2021-12-29,1
3,PTT,36.50,9,0,37.75,38.25,2,37.75,34.00,50.10,0.00,B,2021-12-27,2021-12-29,2
8,RATCH,43.00,0,0,45.25,45.00,-1,44.75,42.00,55.08,0.00,B,2021-12-28,2021-12-29,1
4,ROJNA,6.30,0,0,6.50,6.55,1,6.50,4.22,8.05,0.00,O,2021-12-27,2021-12-29,2
6,SIS,42.00,0,0,43.25,44.00,3,43.25,17.10,43.46,0.00,B,2021-12-28,2021-12-29,1
2,STA,28.00,3,0,29.75,31.25,6,29.75,25.50,41.33,0.00,B,2021-12-20,2021-12-29,7
9,SYNEX,31.25,1,0,33.75,35.25,6,33.75,14.00,29.79,0.00,O,2021-12-28,2021-12-29,1
5,TCAP,35.00,0,0,37.00,37.75,3,37.00,31.50,39.26,0.00,O,2021-12-27,2021-12-29,2


In [7]:
my_array = df_stock.to_numpy()
my_array

array([['DOHOME', 22.9, 1, 0, 23.9, 23.2, -7, 23.1, 12.37, 30.91, 0.0,
        'B', '2021-12-22', '2021-12-29', 5],
       ['GLOBAL', 19.0, 0, 0, 20.0, 20.2, 2, 20.1, 15.69, 26.25, 0.0,
        'B', '2021-12-23', '2021-12-29', 4],
       ['STA', 28.0, 3, 0, 29.75, 31.25, 6, 29.75, 25.5, 41.33, 0.0, 'B',
        '2021-12-20', '2021-12-29', 7],
       ['PTT', 36.5, 9, 0, 37.75, 38.25, 2, 37.75, 34.0, 50.1, 0.0, 'B',
        '2021-12-27', '2021-12-29', 2],
       ['ROJNA', 6.3, 0, 0, 6.5, 6.55, 1, 6.5, 4.22, 8.05, 0.0, 'O',
        '2021-12-27', '2021-12-29', 2],
       ['TCAP', 35.0, 0, 0, 37.0, 37.75, 3, 37.0, 31.5, 39.26, 0.0, 'O',
        '2021-12-27', '2021-12-29', 2],
       ['SIS', 42.0, 0, 0, 43.25, 44.0, 3, 43.25, 17.1, 43.46, 0.0, 'B',
        '2021-12-28', '2021-12-29', 1],
       ['NOBLE', 5.85, 0, 0, 6.1, 6.15, 1, 6.1, 5.9, 6.8, 0.0, 'B',
        '2021-12-28', '2021-12-29', 1],
       ['RATCH', 43.0, 0, 0, 45.25, 45.0, -1, 44.75, 42.0, 55.08, 0.0,
        'B', '2021-12-28', '

In [8]:
tmp = (np.apply_along_axis(number_of_spread, axis=1, arr=my_array))
tmp

array([ -3, -12, -13,  -7,  -5, -11,  -8,  -6,  -8, -16])

In [9]:
df_stock['spd'] = tmp.tolist()
df_stock['spd'] = df_stock['spd'].astype(int)
df_stock.style.format(format_dict)

,name,target,bl,qty,fm_price,to_price,ttl,min,min52,css_price,amount,status,fm_date,to_date,days,spd
0,DOHOME,22.90,1,0,23.90,23.20,-7,23.10,12.37,30.91,0.00,B,2021-12-22,2021-12-29,5,-3
1,GLOBAL,19.00,0,0,20.00,20.20,2,20.10,15.69,26.25,0.00,B,2021-12-23,2021-12-29,4,-12
2,STA,28.00,3,0,29.75,31.25,6,29.75,25.50,41.33,0.00,B,2021-12-20,2021-12-29,7,-13
3,PTT,36.50,9,0,37.75,38.25,2,37.75,34.00,50.10,0.00,B,2021-12-27,2021-12-29,2,-7
4,ROJNA,6.30,0,0,6.50,6.55,1,6.50,4.22,8.05,0.00,O,2021-12-27,2021-12-29,2,-5
5,TCAP,35.00,0,0,37.00,37.75,3,37.00,31.50,39.26,0.00,O,2021-12-27,2021-12-29,2,-11
6,SIS,42.00,0,0,43.25,44.00,3,43.25,17.10,43.46,0.00,B,2021-12-28,2021-12-29,1,-8
7,NOBLE,5.85,0,0,6.10,6.15,1,6.10,5.90,6.80,0.00,B,2021-12-28,2021-12-29,1,-6
8,RATCH,43.00,0,0,45.25,45.00,-1,44.75,42.00,55.08,0.00,B,2021-12-28,2021-12-29,1,-8
9,SYNEX,31.25,1,0,33.75,35.25,6,33.75,14.00,29.79,0.00,O,2021-12-28,2021-12-29,1,-16


In [10]:
cols = 'name spd target bl qty fm_price to_price ttl min min52 css_price status amount fm_date to_date days'.split()
df_stock[cols].style.format(format_dict)

,name,spd,target,bl,qty,fm_price,to_price,ttl,min,min52,css_price,status,amount,fm_date,to_date,days
0,DOHOME,-3,22.90,1,0,23.90,23.20,-7,23.10,12.37,30.91,B,0.00,2021-12-22,2021-12-29,5
1,GLOBAL,-12,19.00,0,0,20.00,20.20,2,20.10,15.69,26.25,B,0.00,2021-12-23,2021-12-29,4
2,STA,-13,28.00,3,0,29.75,31.25,6,29.75,25.50,41.33,B,0.00,2021-12-20,2021-12-29,7
3,PTT,-7,36.50,9,0,37.75,38.25,2,37.75,34.00,50.10,B,0.00,2021-12-27,2021-12-29,2
4,ROJNA,-5,6.30,0,0,6.50,6.55,1,6.50,4.22,8.05,O,0.00,2021-12-27,2021-12-29,2
5,TCAP,-11,35.00,0,0,37.00,37.75,3,37.00,31.50,39.26,O,0.00,2021-12-27,2021-12-29,2
6,SIS,-8,42.00,0,0,43.25,44.00,3,43.25,17.10,43.46,B,0.00,2021-12-28,2021-12-29,1
7,NOBLE,-6,5.85,0,0,6.10,6.15,1,6.10,5.90,6.80,B,0.00,2021-12-28,2021-12-29,1
8,RATCH,-8,43.00,0,0,45.25,45.00,-1,44.75,42.00,55.08,B,0.00,2021-12-28,2021-12-29,1
9,SYNEX,-16,31.25,1,0,33.75,35.25,6,33.75,14.00,29.79,O,0.00,2021-12-28,2021-12-29,1


In [11]:
buy_alert = df_stock[cols]
buy_alert.set_index(['name'],inplace=True)
buy_alert.sort_values(['spd'],ascending=[False]).style.format(format_dict)

,spd,target,bl,qty,fm_price,to_price,ttl,min,min52,css_price,status,amount,fm_date,to_date,days
name,,,,,,,,,,,,,,,
DOHOME,-3,22.90,1,0,23.90,23.20,-7,23.10,12.37,30.91,B,0.00,2021-12-22,2021-12-29,5
ROJNA,-5,6.30,0,0,6.50,6.55,1,6.50,4.22,8.05,O,0.00,2021-12-27,2021-12-29,2
NOBLE,-6,5.85,0,0,6.10,6.15,1,6.10,5.90,6.80,B,0.00,2021-12-28,2021-12-29,1
PTT,-7,36.50,9,0,37.75,38.25,2,37.75,34.00,50.10,B,0.00,2021-12-27,2021-12-29,2
SIS,-8,42.00,0,0,43.25,44.00,3,43.25,17.10,43.46,B,0.00,2021-12-28,2021-12-29,1
RATCH,-8,43.00,0,0,45.25,45.00,-1,44.75,42.00,55.08,B,0.00,2021-12-28,2021-12-29,1
TCAP,-11,35.00,0,0,37.00,37.75,3,37.00,31.50,39.26,O,0.00,2021-12-27,2021-12-29,2
GLOBAL,-12,19.00,0,0,20.00,20.20,2,20.10,15.69,26.25,B,0.00,2021-12-23,2021-12-29,4
STA,-13,28.00,3,0,29.75,31.25,6,29.75,25.50,41.33,B,0.00,2021-12-20,2021-12-29,7


In [12]:
file_name = 'buy-alert.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
data_file, output_file, box_file

('../data/buy-alert.csv',
 '\\Users\\User\\iCloudDrive\\buy-alert.csv',
 '\\Users\\User\\Dropbox\\buy-alert.csv')

In [13]:
buy_alert.sort_values(['spd','name'],ascending=[False,True]).to_csv(output_file)
buy_alert.sort_values(['spd','name'],ascending=[False,True]).to_csv(data_file)
buy_alert.sort_values(['spd','name'],ascending=[False,True]).to_csv(box_file)